# Языковые модели на марковских цепях

Обучим символьные модели для генерации имён динозавров.

### Данные: имена динозавров

In [0]:
!wget https://raw.githubusercontent.com/artemovae/ML-for-compling/master/2018/dinos.txt

--2020-04-01 17:49:05--  https://raw.githubusercontent.com/artemovae/ML-for-compling/master/2018/dinos.txt
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 151.101.0.133, 151.101.64.133, 151.101.128.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|151.101.0.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 19909 (19K) [text/plain]
Saving to: ‘dinos.txt’

dinos.txt           100%[===================>]  19.44K  --.-KB/s    in 0.007s  

2020-04-01 17:49:05 (2.78 MB/s) - ‘dinos.txt’ saved [19909/19909]



In [0]:
!cat dinos.txt | wc -l

1535


## Марковская цепь


Снижаем регистр, добавляем символы начала и конца строки.

In [0]:
names = ['<' + name.strip().lower() + '>' for name in open('dinos.txt').readlines()]
names[:10]

['<aachenosaurus>',
 '<aardonyx>',
 '<abdallahsaurus>',
 '<abelisaurus>',
 '<abrictosaurus>',
 '<abrosaurus>',
 '<abydosaurus>',
 '<acanthopholis>',
 '<achelousaurus>',
 '<acheroraptor>']

In [0]:
import nltk

Вычислим частоту каждого символа в корпусе имен динозавров:

In [0]:
chars = [char for name in names for char in name]


In [0]:
freq = nltk.FreqDist(chars)

In [0]:
freq

FreqDist({'<': 1536,
          '>': 1536,
          'a': 2487,
          'b': 171,
          'c': 539,
          'd': 341,
          'e': 913,
          'f': 37,
          'g': 360,
          'h': 548,
          'i': 944,
          'j': 55,
          'k': 141,
          'l': 617,
          'm': 328,
          'n': 1081,
          'o': 1710,
          'p': 552,
          'q': 23,
          'r': 1704,
          's': 2285,
          't': 852,
          'u': 2123,
          'v': 111,
          'w': 41,
          'x': 85,
          'y': 266,
          'z': 60})

In [0]:
print(list(freq.keys()))

['j', 'u', 'g', 'n', 'y', 'r', 'l', 'o', 's', 'v', 'c', 'z', 'i', 'k', 'q', 'b', '>', 'd', '<', 'p', 't', 'm', 'w', 'a', 'f', 'x', 'e', 'h']


In [0]:
freq.most_common(10)

[('a', 2487),
 ('s', 2285),
 ('u', 2123),
 ('o', 1710),
 ('r', 1704),
 ('<', 1536),
 ('>', 1536),
 ('n', 1081),
 ('i', 944),
 ('e', 913)]

Определим функцию, чтобы оценить веротность символа:

In [0]:
l = sum([freq[char] for char in freq])
def unigram_prob(char):
    return freq[char] / l

In [0]:
print('p(a) = %1.4f' %unigram_prob('s'))

p(a) = 0.1065


### Биграммная модель

Вычислим условную вероятность каждого символа в зависимости от того, какой символ стоял на предыдущей позиции.

In [0]:
bigrams = nltk.bigrams(chars) # список пар идущих друг за другом символов

In [0]:
cfreq = nltk.ConditionalFreqDist(nltk.bigrams(chars))

Оценим условные вероятности с помощью MLE.

In [0]:
# просто подсчитавыем, сколько раз какой символ встретился после каждого из символов
cprob = nltk.ConditionalProbDist(cfreq, nltk.MLEProbDist)

In [0]:
print('p(a a) = %1.4f' %cprob['a'].prob('a'))
print('p(a b) = %1.4f' %cprob['a'].prob('b'))
print('p(a u) = %1.4f' %cprob['a'].prob('u'))

p(a a) = 0.0044
p(a b) = 0.0097
p(a u) = 0.3181


In [0]:
cprob['a'].generate() # просим породить следующий символ

'n'

### Задание 1

Напишите функцию, которая генерирует имя динозавра **фиксированной** длины. Используйте '<' как начальный символ.

'<hanyran><>'

### Задание 2

1) Напишите функцию, которая генерирует имя динозавра любой дины.

2) сделайте возможность при вызове функции самостоятельно подавать начало имени динозавра.

### Задание 3

Напишите функцию, которая принимает имя динозавра (строку) и возвращает вероятность этого имени на основании символов, из которых оно состоит.  

### Задание 4

Напишите функцию, которая принимает два имени динозавра и возвращает из них самое вероятное.

### Панграмы

Давайте напишем функцию, которая генерирует панграмы -- имена динозавров, не содержащие одинаковых букв.

## На уровне слов

Давайте теперь возьмём много текстов -- например, [книжку по программированию в txt](https://drive.google.com/open?id=17oT5ZEmUcSS_C_0eS6jZBMhn-q2YipaK) -- и попробуем сделать то же самое, но со словами.

In [0]:
# для колаба
from google.colab import files
uploaded = files.upload()

Saving coding.txt to coding.txt


In [0]:
from nltk import word_tokenize, sent_tokenize

In [0]:
with open('coding.txt', encoding='utf-8') as f:
    text = f.read()

In [0]:
print(text[:200])

 Кормен (Renee Connen).

Предисловие
Как компьютеры решают задачи? Как ваш маленький GPS в считанные секунды на­
ходит самый быстрый пуrь из несметного множества возможных маршруrов? Когда вы
покупае
